--IrisTraderBot--


In [ ]:
# Cell 1: All imports and configuration variables.

import time
import datetime
from irismt5linux import MetaTrader5
import pandas as pd
import concurrent.futures # Still imported but not used directly for parallel fetching anymore
import os
import numpy as np
from datetime import datetime, timedelta

# --- Paths ---
MT5_PATH = 'Z:/home/r00tz/.mt5/drive_c/Program Files/MetaTrader 5/terminal64.exe'

# --- Configuration ---
HOST = 'localhost'
PORT = 18812

# --- Credentials ---
LOGIN_ID = 93786391 # Updated to OANDA demo login
PASSWORD = 'R-6iKkYf' # IMPORTANT: Replace with your actual OANDA demo password
SERVER = 'MetaQuotes-Demo' # Updated to OANDA demo server

# --- Target Symbols ---
# Adjusted to reflect working OANDA demo symbols and remove problematic ones
SYMBOLS_TO_TRADE = ["EURUSD", "USDJPY", "GBPUSD", "AUDUSD", "GBPJPY",
                    "USDCHF", "USDCAD", "EURJPY", "AUDJPY", "CADJPY"]
CACHE_DIR = '/home/r00tz/Documents/Codes/IrisTrader/cache/pairs'

# --- Global Trading Parameters ---
MAX_LOSS_PER_0_01_LOT = -2.00 # Max loss in USD for a 0.01 lot trade
MIN_PROFIT_PER_0_01_LOT = 0.50 # Min profit in USD for a 0.01 lot trade

# Stop Loss and Take Profit points (these are initial values for broker-side SL/TP)
DEFAULT_SL_POINTS = 70
DEFAULT_TP_POINTS = 100

# RSI thresholds for signal filtering
RSI_BUY_THRESHOLD = 70   # Don't buy if RSI is above this (overbought)
RSI_SELL_THRESHOLD = 30  # Don't sell if RSI is below this (oversold)

# SMA periods for indicator calculations
SMA_FAST_PERIOD_M1 = 10
SMA_SLOW_PERIOD_M1 = 20
SMA_SLOW_PERIOD_H1 = 50 

# MACD and ATR Parameters
MACD_FAST_PERIOD = 12
MACD_SLOW_PERIOD = 26
MACD_SIGNAL_PERIOD = 9
ATR_PERIOD = 14
ATR_MULTIPLIER = 1.5 # Multiplier for ATR-based stop loss (e.g., 1.5 * ATR)
MIN_ATR_VALUE = 0.00005 # Minimum ATR value to consider for trading (filters out extremely flat markets)

# Trade volume per order
DEFAULT_TRADE_VOLUME = 0.30 

# Loop Control Parameter for faster checks
BOT_LOOP_INTERVAL_SECONDS = 10 # How often the main bot loop runs (e.g., every 10 seconds)

In [ ]:
# Cell 2: All of our helper functions.
# Run this cell after Cell 1.

import pandas as pd
import pandas_ta as ta  # <-- NEW: Import the pandas-ta library
import numpy as np
import time
import os
from datetime import datetime, timedelta
from irismt5linux import MetaTrader5

# This cell relies on global constants defined in Cell 1.

def display_account_info(mt5_client):
    """
    Retrieves and displays key account information in a clean format.
    """
    print("---")
    print("Requesting Account Information...")
    time.sleep(1) 
    info = mt5_client.account_info()
    if info:
        print("\n--- Account Details ---")
        print(f"Login:            {info.login}")
        print(f"Name:             {info.name}")
        print(f"Broker:           {info.server}")
        print(f"Currency:         {info.currency}")
        print("-----------------------")
        print(f"Balance:          {info.balance:.2f}")
        print(f"Equity:           {info.equity:.2f}")
        print(f"Margin:           {info.margin:.2f}")
        print(f"Margin Free:      {info.margin_free:.2f}")
        print(f"Margin Level:     {info.margin_level:.2f}%")
        print("-----------------------")
        return True
    else:
        print(f"\nCould not retrieve account information. MT5 Error: {mt5_client.last_error()}")
        return False

def connect_and_login():
    """
    Handles the entire connection and login process.
    """
    client = None
    try:
        client = MetaTrader5(host=HOST, port=PORT)
        print("Successfully connected to the mt5linux proxy server.")
    except Exception as e:
        print(f"Failed to create client object: {e}")
        return None

    print('---')
    print('Attempting to launch and log in to MT5 terminal...')

    if not client.initialize(path=MT5_PATH, login=LOGIN_ID, password=PASSWORD, server=SERVER):
        print(f"IPC connection to terminal FAILED. Error code: {client.last_error()}")
        client.shutdown()
        return None
    
    print("IPC connection to terminal SUCCEEDED.")
    print("Waiting for 5 seconds for broker login to complete...")
    time.sleep(5)
    return client

def get_current_prices(mt5_client, symbols):
    """
    Fetches and displays the current bid/ask price for a list of symbols.
    """
    print("\n--- Current Market Prices ---")
    for symbol in symbols:
        tick = mt5_client.symbol_info_tick(symbol)
        if tick:
            print(f"{symbol:<12} Bid: {tick.bid:<8} Ask: {tick.ask:<8}")
        else:
            print(f"Could not get tick for {symbol}. Is it enabled in Market Watch?")
    print("----------------------------")

def _fetch_live_historical_data(mt5_client, symbol, timeframe, num_bars=100):
    """
    Fetches historical OHLC data LIVE from MT5 and includes a check for the last MT5 error.
    """
    print(f"--- Fetching LIVE Historical Data for {symbol} ({get_timeframe_string(timeframe)}) ---")
    rates = mt5_client.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
    
    if not isinstance(rates, (list, tuple)):
        print(f"❌ ERROR: Expected a list of rate data, but received type {type(rates)}.")
        print(f"   --> Raw data received: {rates}")
        return None

    # --- START OF THE FIX ---
    # If we received an empty list, ask MT5 for the specific reason.
    if len(rates) == 0:
        error_code, error_message = mt5_client.last_error()
        print(f"❌ Failed to get historical data for {symbol}. Received 0 bars.")
        print(f"   MT5 Last Error: Code {error_code} - {error_message}")
        return None
    # --- END OF THE FIX ---

    df = pd.DataFrame(rates)
    df['time'] = pd.to_numeric(df['time'], errors='coerce')
    df.dropna(subset=['time'], inplace=True)
    df['time'] = df['time'].astype(int)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    print(f"✅ Successfully retrieved and formatted {len(df)} bars of LIVE data for {symbol}.")
    return df


def get_timeframe_string(timeframe_int):
    """Converts an MT5 timeframe constant to a string like 'H1', 'M5', etc."""
    timeframe_map = {
        MetaTrader5.TIMEFRAME_M1: 'M1',
        MetaTrader5.TIMEFRAME_M5: 'M5',
        MetaTrader5.TIMEFRAME_M15: 'M15',
        MetaTrader5.TIMEFRAME_M30: 'M30',
        MetaTrader5.TIMEFRAME_H1: 'H1',
        MetaTrader5.TIMEFRAME_H4: 'H4',
        MetaTrader5.TIMEFRAME_D1: 'D1',
    }
    return timeframe_map.get(timeframe_int, str(timeframe_int))

def get_data_from_cache(mt5_client, symbol, timeframe, num_bars):
    """
    Intelligent caching system that tries to load from cache first, then falls back
    to live data if the cache is stale or non-existent.
    """
    # This function remains as it was, as its logic is sound.
    # We'll skip re-pasting its large block of code for brevity, assuming you still have it.
    # If not, the essential logic is to use _fetch_live_historical_data and save/load from CSV.
    # For a complete script, we'll assume the live fetch is always used for simplicity here.
    return _fetch_live_historical_data(mt5_client, symbol, timeframe, num_bars)


def calculate_indicators(df, sma_fast=None, sma_slow=None, rsi_period=14, macd_fast=None, macd_slow=None, macd_signal=None, atr_period=None):
    """
    Calculates technical indicators using pandas-ta for efficiency and power.
    Now includes ADX for trend strength analysis.
    """
    if df is None or df.empty:
        print("DataFrame is empty, cannot calculate indicators.")
        return df

    print(f"Calculating indicators for {len(df)} bars of data...")
    
    # Use the df.ta extension provided by pandas-ta for easy indicator calculation.
    if sma_fast: df.ta.sma(length=sma_fast, append=True, col_names=(f'SMA_{sma_fast}',))
    if sma_slow: df.ta.sma(length=sma_slow, append=True, col_names=(f'SMA_{sma_slow}',))
    if rsi_period: df.ta.rsi(length=rsi_period, append=True)
    if macd_fast and macd_slow and macd_signal:
        df.ta.macd(fast=macd_fast, slow=macd_slow, signal=macd_signal, append=True)
    if atr_period: df.ta.atr(length=atr_period, append=True)
    
    # Add ADX for our new regime filter
    df.ta.adx(append=True)
    
    df.dropna(inplace=True)
    print("Indicators calculated successfully, including ADX.")
    return df

def place_market_order(mt5_client, symbol, volume, is_buy):
    """
    Prepares and sends a market order, ensuring SL/TP distances respect the broker's minimum StopsLevel
    and are calculated from the correct side of the bid/ask spread.
    Uses default SL/TP points from global config.
    """
    sl_points = DEFAULT_SL_POINTS
    tp_points = DEFAULT_TP_POINTS
    magic_number = 12345
    
    order_type_str_log = 'BUY' if is_buy else 'SELL'
    print(f"\n--- Preparing {order_type_str_log} order for {symbol} (Volume: {volume}) ---")
    
    symbol_info = mt5_client.symbol_info(symbol)
    if not symbol_info:
        print(f"Error: Could not get symbol info for {symbol}. Cannot place order.")
        return None
        
    tick = mt5_client.symbol_info_tick(symbol)
    if not tick:
        print(f"Error: Could not get tick for {symbol}. Cannot place order.")
        return None

    # Get the broker's required minimum stop distance in points (StopsLevel)
    stops_level = symbol_info.trade_stops_level
    print(f"Broker's required minimum stop distance (StopsLevel) for {symbol}: {stops_level} points.")

    if sl_points < stops_level:
        print(f"⚠️ WARNING: Your SL of {sl_points} points is less than broker's required minimum of {stops_level}.")
        sl_points = stops_level
        print(f"   --> SL points automatically adjusted to {sl_points}.")
    if tp_points < stops_level:
        print(f"⚠️ WARNING: Your TP of {tp_points} points is less than broker's required minimum of {stops_level}.")
        tp_points = stops_level
        print(f"   --> TP points automatically adjusted to {tp_points}.")

    # --- CRITICAL FIX FOR SPREAD ---
    point = symbol_info.point
    digits = symbol_info.digits
    sl, tp, price = 0.0, 0.0, 0.0

    if is_buy:
        price = tick.ask
        sl = tick.bid - (sl_points * point)
        tp = tick.ask + (tp_points * point)
    else: # is_buy is False (SELL)
        price = tick.bid
        sl = tick.ask + (sl_points * point) # SL for a SELL is based on ASK
        tp = tick.bid - (tp_points * point)
        
    sl, tp = round(sl, digits), round(tp, digits)
    
    print(f"Entry Price: {price}, Current Ask: {tick.ask}, Current Bid: {tick.bid}")
    print(f"Calculated SL: {sl}, Calculated TP: {tp}")

    request = {
        "action": mt5_client.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(volume),
        "type": mt5_client.ORDER_TYPE_BUY if is_buy else mt5_client.ORDER_TYPE_SELL,
        "price": price,
        "sl": sl,
        "tp": tp,
        "deviation": 20,
        "magic": magic_number,
        "comment": "IrisTrader Order",
        "type_time": mt5_client.ORDER_TIME_GTC,
        "type_filling": mt5_client.ORDER_FILLING_FOK,
    }
    
    result = mt5_client.order_send(request)
    
    if result is None:
        print(f"❌ order_send returned NONE! MT5 Last Error: {mt5_client.last_error()}")
        return None
    
    if result.retcode == mt5_client.TRADE_RETCODE_DONE:
        print(f"✅ Order sent successfully! Order ticket: {result.order}, Price: {result.price:.{digits}f}")
    else:
        print(f"❌ Order failed! Retcode: {result.retcode} - {result.comment}")
        if hasattr(result, 'request'): print(f"   Request: {result.request}")
    return result

def close_position(mt5_client, position):
    """
    Closes a specific open position.
    """
    print(f"--- Preparing to CLOSE position for {position.symbol} (Ticket: {position.ticket}, Profit: {position.profit:.2f}) ---")
    
    tick = mt5_client.symbol_info_tick(position.symbol)
    if not tick:
        print(f"Error: Could not get tick for {position.symbol}. Cannot close order.")
        return None

    # Position type 0 is BUY, 1 is SELL. Order type is the opposite action.
    close_order_type = mt5_client.ORDER_TYPE_SELL if position.type == 0 else mt5_client.ORDER_TYPE_BUY
    price = tick.bid if position.type == 0 else tick.ask

    close_request = {
        "action": mt5_client.TRADE_ACTION_DEAL,
        "symbol": position.symbol,
        "volume": float(position.volume),
        "type": close_order_type,
        "position": position.ticket,
        "price": price,
        "deviation": 20,
        "magic": position.magic,
        "comment": "IrisTrader Close",
        "type_time": mt5_client.ORDER_TIME_GTC,
        "type_filling": mt5_client.ORDER_FILLING_FOK,
    }
    
    result = mt5_client.order_send(close_request)
    
    if result and result.retcode == mt5_client.TRADE_RETCODE_DONE:
        print(f"✅ Position {position.ticket} closed successfully!")
    else:
        retcode = result.retcode if result else 'None'
        comment = result.comment if result else 'N/A'
        print(f"❌ Position close failed! Retcode: {retcode} - Comment: {comment}")
    return result

def generate_signals(df, rsi_buy_threshold, rsi_sell_threshold, macd_check, atr_check):
    """
    Generates trading signals based on SMA Alignment, with optional MACD and ATR filters.
    """
    # Using pandas-ta column names, e.g., 'SMA_10', 'SMA_20'
    fast_sma_col = f"SMA_{SMA_FAST_PERIOD_M1}"
    slow_sma_col = f"SMA_{SMA_SLOW_PERIOD_M1}"
    rsi_col = f"RSI_{14}" # Assuming default RSI period
    macd_col = f"MACD_{MACD_FAST_PERIOD}_{MACD_SLOW_PERIOD}_{MACD_SIGNAL_PERIOD}"
    macd_signal_col = f"MACDs_{MACD_FAST_PERIOD}_{MACD_SLOW_PERIOD}_{MACD_SIGNAL_PERIOD}"
    atr_col = f"ATRr_{ATR_PERIOD}"

    required_cols = [fast_sma_col, slow_sma_col, rsi_col]
    if not all(col in df.columns for col in required_cols):
        print(f"DataFrame missing required columns for signal generation. Needed: {required_cols}")
        df['position'] = 0
        return df

    print("Generating filtered signals (SMA Alignment + RSI + MACD/ATR filters)...")
    
    buy_signals = (df[fast_sma_col] > df[slow_sma_col]) & (df[rsi_col] < rsi_buy_threshold)
    sell_signals = (df[fast_sma_col] < df[slow_sma_col]) & (df[rsi_col] > rsi_sell_threshold)

    if macd_check and macd_col in df.columns and macd_signal_col in df.columns:
        buy_signals &= (df[macd_col] > df[macd_signal_col])
        sell_signals &= (df[macd_col] < df[macd_signal_col])
    if atr_check and atr_col in df.columns:
        buy_signals &= (df[atr_col] > MIN_ATR_VALUE)
        sell_signals &= (df[atr_col] > MIN_ATR_VALUE)

    df['position'] = np.where(buy_signals, 1, np.where(sell_signals, -1, 0))
    print("Filtered signals generated.")
    return df

def calculate_sleep_time(interval_seconds):
    """
    Helper to print a sleep message. The bot loop is now fixed interval.
    """
    # This function is simpler now as the main loop uses a fixed sleep interval.
    print(f"\nAnalysis cycle complete. Sleeping for {interval_seconds} seconds...")
    time.sleep(interval_seconds)

def check_symbol_availability(mt5_client, symbol):
    """
    Checks if a symbol exists, is visible in Market Watch, and has recent ticks.
    Returns True if the symbol is available for trading, False otherwise.
    """
    print(f"--- Performing pre-flight check for {symbol} ---")
    
    # 1. Check if symbol exists
    symbol_info = mt5_client.symbol_info(symbol)
    if not symbol_info:
        print(f"❌ CHECK FAILED: Symbol {symbol} not found on the broker's server.")
        return False

    # 2. Check if symbol is visible in Market Watch
    if not symbol_info.visible:
        print(f"❌ CHECK FAILED: Symbol {symbol} is NOT VISIBLE in Market Watch.")
        print("   ACTION: Please open your MT5 terminal, right-click in the Market Watch window, choose 'Symbols', find this symbol, and double-click it to enable it (it should turn yellow).")
        return False
        
    # 3. Check for recent quotes
    # A time of 0 means no quotes have been received yet.
    if symbol_info.time == 0:
        print(f"❌ CHECK FAILED: Symbol {symbol} is visible but has not received any quotes yet. It may be inactive.")
        return False
        
    print(f"✅ CHECK PASSED: Symbol {symbol} is active and ready.")
    return True

In [ ]:
# Cell 3: A "Minimal Viable Bot" for a final diagnostic test.

def run_minimal_bot():
    """
    This is the simplest possible bot loop. It connects once, then loops
    through symbols trying to fetch data. Its only purpose is to see
    if the core loop itself is the problem.
    """
    print("🤖 Minimal Viable Bot (Final Test) initializing...")
    
    # Connect once at the very beginning
    client = connect_and_login()
    if not client:
        print("Initial connection failed. Exiting.")
        return

    print("\n--- Starting the minimal analysis loop ---")
    try:
        # We will just run the loop 5 times for this test.
        for i in range(5):
            print("\n" + "="*70)
            print(f"Executing Test Loop Iteration #{i + 1}")
            print("="*70)
            
            for symbol in SYMBOLS_TO_TRADE:
                print(f"--- Analyzing {symbol} ---")
                
                # Directly call the data fetching function
                # We are requesting only 10 bars for this test
                h1_data = _fetch_live_historical_data(client, symbol, MetaTrader5.TIMEFRAME_H1, 10)
                
                if h1_data is not None and not h1_data.empty:
                    print(f"   ✅✅✅ SUCCESS! Got {len(h1_data)} bars for {symbol}.")
                else:
                    print(f"   ❌❌❌ FAILED. Got 0 bars or None for {symbol}.")
                
                # A delay to be kind to the API
                time.sleep(5)
            
            print("\nCycle complete. Sleeping for 10 seconds...")
            time.sleep(10)

    except KeyboardInterrupt:
        print("\nBot interrupted by user.")
    except Exception as e:
        print(f"\nAn unexpected error occurred in the minimal loop: {e}")
    finally:
        print("--- Minimal test finished. Shutting down connection. ---")
        if client:
            client.shutdown()

# --- Run the Minimal Bot ---
run_minimal_bot()

In [ ]:
# Cell 3: The final bot loop, now with an ADX-based market regime filter to make trading decisions smarter.
# for active trade management and improved responsiveness.

def run_trading_bot():
    """
    The main trading bot loop. It continuously analyzes market conditions,
    manages open positions, and opens new trades based on an intelligent, adaptive strategy.
    """
    print("🤖 Iris Active Trader Bot v5.0 (with ADX Regime Filter) initializing...")
    
    iris_mt5_client = connect_and_login()
    if not iris_mt5_client:
        print("Failed to connect or log in to MT5. Exiting bot.")
        return

    if not display_account_info(iris_mt5_client):
        print("Failed to retrieve account info. Exiting bot.")
        iris_mt5_client.shutdown()
        return

    # Define the threshold for what we consider a "strong" trend.
    ADX_STRONG_TREND_THRESHOLD = 25

    try:
        last_entry_signal_check_time_m1 = None

        while True:
            current_loop_time = datetime.now()
            print("\n" + "="*70)
            print(f"Executing new analysis cycle at: {current_loop_time.strftime('%Y-%m-%d %H:%M:%S')}")
            print("="*70)
            
            # # --- START: Handle Open Positions Actively and Frequently ---
            # open_positions = iris_mt5_client.positions_get()
            # positions_map = {p.symbol: p for p in open_positions} if open_positions else {}
            # print(f"Currently open positions: {list(positions_map.keys()) if positions_map else 'None'}")

            # tradable_symbols_for_management = []
            # for symbol in SYMBOLS_TO_TRADE:
            #     symbol_info = iris_mt5_client.symbol_info(symbol)
            #     if symbol_info and symbol_info.visible:
            #         tradable_symbols_for_management.append(symbol)
            
            # for symbol in tradable_symbols_for_management:
            #     if symbol in positions_map:
            #         position = positions_map[symbol]
            #         print(f"\nManaging open position for {symbol}: Type {'BUY' if position.type == 0 else 'SELL'}, Profit: {position.profit:.2f}, Volume: {position.volume:.2f}")

            #         scaling_factor = position.volume / 0.01
            #         current_max_loss = MAX_LOSS_PER_0_01_LOT * scaling_factor
            #         current_min_profit = MIN_PROFIT_PER_0_01_LOT * scaling_factor

            #         tick = iris_mt5_client.symbol_info_tick(symbol)
            #         if not tick:
            #             print(f"Warning: Could not get tick for {symbol}. Skipping position management for this cycle.")
            #             continue

            #         # PRIORITY 1 & 2: Profit/Loss-based exits (unchanged)
            #         if position.profit <= current_max_loss:
            #             print(f"🚨 HARD STOP HIT: Profit at {position.profit:.2f} is below dynamic max loss of {current_max_loss:.2f}. Forcing closure.")
            #             close_position(iris_mt5_client, position)
            #             continue 
            #         if position.profit >= current_min_profit:
            #             print(f"💰 PROFIT TARGET HIT: Profit at {position.profit:.2f} is above dynamic min profit of {current_min_profit:.2f}. Closing position.")
            #             close_position(iris_mt5_client, position)
            #             continue
                    
            #         # PRIORITY 3 & 4: Strategic exits based on H1 trend (unchanged)
            #         h1_data_for_exit = get_data_from_cache(iris_mt5_client, symbol, MetaTrader5.TIMEFRAME_H1, 100)
            #         if h1_data_for_exit is None or h1_data_for_exit.empty:
            #             print(f"Warning: Could not get H1 data for strategic exit for {symbol}.")
            #         else:
            #             # Ensure we get the correct column name from pandas-ta (e.g., SMA_50)
            #             h1_data_for_exit = calculate_indicators(h1_data_for_exit, sma_slow=SMA_SLOW_PERIOD_H1)
            #             if not h1_data_for_exit.empty:
            #                 last_h1_candle_for_exit = h1_data_for_exit.iloc[-1]
            #                 # Assuming pandas-ta creates a column like 'SMA_50'
            #                 slow_sma_col_name = f'SMA_{SMA_SLOW_PERIOD_H1}' 
            #                 if slow_sma_col_name in last_h1_candle_for_exit:
            #                     current_h1_trend = "UP" if last_h1_candle_for_exit['close'] > last_h1_candle_for_exit[slow_sma_col_name] else "DOWN"
                                
            #                     # PRIORITY 4: STRATEGIC EXIT (Trend Reversal on H1)
            #                     if (position.type == MetaTrader5.POSITION_TYPE_BUY and current_h1_trend == "DOWN") or \
            #                        (position.type == MetaTrader5.POSITION_TYPE_SELL and current_h1_trend == "UP"):
            #                         print(f"Trend Reversal! Closing position for {symbol} because trade is against H1 trend.")
            #                         close_position(iris_mt5_client, position)
            #                         continue
            #                 else:
            #                     print(f"Warning: Could not find SMA column '{slow_sma_col_name}' for exit analysis.")
            #             else:
            #                 print(f"Warning: H1 data became empty after indicator calculation for strategic exit of {symbol}.")
                    
            #         print(f"Decision: Hold. No active exit conditions met for {symbol}.")
            # # --- END: Handle Open Positions ---

            positions_map = {}

            # --- START: Intelligent Entry Signal Generation (Candle-Aligned) ---
            tradable_symbols_for_entry = [s for s in SYMBOLS_TO_TRADE if s not in positions_map]
            if not tradable_symbols_for_entry:
                print("\n--- All symbols have open positions or are untradable. No new entries considered. ---")
            else:
                current_minute_start = current_loop_time.replace(second=0, microsecond=0)
                should_check_m1 = False
                if last_entry_signal_check_time_m1 is None or current_minute_start > last_entry_signal_check_time_m1:
                    should_check_m1 = True
                    last_entry_signal_check_time_m1 = current_minute_start

                if should_check_m1:
                    print("\n--- Considering New Entries (New M1 Candle) ---")
                    for symbol in tradable_symbols_for_entry:
                        print(f"\n--- Analyzing {symbol} for new entry ---")

                        # --- USE OUR NEW PRE-FLIGHT CHECK ---
                        if not check_symbol_availability(iris_mt5_client, symbol):
                            print(f"--- Skipping {symbol} due to availability issues. ---")
                            continue # Move to the next symbol

                        # 1. Get H1 data to determine the market regime (trend and strength)
                        h1_data = get_data_from_cache(iris_mt5_client, symbol, MetaTrader5.TIMEFRAME_H1, 10) # More bars for better ADX
                        if h1_data is None or h1_data.empty:
                            print(f"Skipping {symbol}: Could not get H1 data for regime analysis.")
                            continue
                        
                        h1_data = calculate_indicators(h1_data, sma_slow=SMA_SLOW_PERIOD_H1) # This will now add ADX
                        if h1_data.empty:
                            print(f"Skipping {symbol}: H1 data became empty after indicator calculation.")
                            continue

                        last_h1_candle = h1_data.iloc[-1]
                        slow_sma_col_name_h1 = f'SMA_{SMA_SLOW_PERIOD_H1}'
                        adx_col_name = 'ADX_14' # Default name from pandas-ta

                        if slow_sma_col_name_h1 not in last_h1_candle or adx_col_name not in last_h1_candle:
                            print(f"Skipping {symbol}: Missing required H1 indicator columns ({slow_sma_col_name_h1}, {adx_col_name}).")
                            continue

                        main_trend = "UP" if last_h1_candle['close'] > last_h1_candle[slow_sma_col_name_h1] else "DOWN"
                        adx_value = last_h1_candle.get(adx_col_name, 0)
                        
                        print(f"H1 Market Analysis for {symbol}: Trend is {main_trend}, Strength (ADX) is {adx_value:.2f}")

                        # 2. THE NEW "BRAIN": The ADX Regime Filter
                        if adx_value > ADX_STRONG_TREND_THRESHOLD:
                            print(f"✅ Regime Active: Strong trend detected. Looking for M1 entries aligned with H1 trend.")
                            
                            m1_data = get_data_from_cache(iris_mt5_client, symbol, MetaTrader5.TIMEFRAME_M1, 100)
                            if m1_data is None or m1_data.empty:
                                print(f"   --> Skipping {symbol}: Could not get M1 data for entry signal.")
                                continue

                            m1_data = calculate_indicators(m1_data, sma_fast=SMA_FAST_PERIOD_M1, sma_slow=SMA_SLOW_PERIOD_M1, macd_fast=MACD_FAST_PERIOD, macd_slow=MACD_SLOW_PERIOD, macd_signal=MACD_SIGNAL_PERIOD, atr_period=ATR_PERIOD)
                            if m1_data.empty:
                                print(f"   --> Skipping {symbol}: M1 data became empty after indicator calculation.")
                                continue
                            
                            # Using pandas-ta names here for clarity
                            m1_data_with_signals = generate_signals(m1_data, rsi_buy_threshold=RSI_BUY_THRESHOLD, rsi_sell_threshold=RSI_SELL_THRESHOLD, macd_check=True, atr_check=True)
                            last_m1_signal = m1_data_with_signals['position'].iloc[-1]

                            # 3. Firing logic: Only trade if M1 signal aligns with strong H1 trend
                            if last_m1_signal == 1 and main_trend == "UP":
                                print(f"   --> FIRING BUY SIGNAL (M1 signal aligns with H1 UP-Trend)!")
                                place_market_order(iris_mt5_client, symbol, DEFAULT_TRADE_VOLUME, is_buy=True)

                            elif last_m1_signal == -1 and main_trend == "DOWN":
                                print(f"   --> FIRING SELL SIGNAL (M1 signal aligns with H1 DOWN-Trend)!")
                                place_market_order(iris_mt5_client, symbol, DEFAULT_TRADE_VOLUME, is_buy=False)

                            else:
                                print(f"   --> Holding. No M1 entry signal, or signal ({last_m1_signal}) conflicts with H1 trend.")

                        else: # ADX is below the threshold
                            print(f"❌ Regime Inactive: Market is choppy or trend is too weak (ADX below {ADX_STRONG_TREND_THRESHOLD}).")
                            print(f"   --> No new entries will be considered for {symbol} to protect capital.")
                        print("--- Short delay before next symbol ---")
                        time.sleep(0.5) # Sleep for 0.5 second
                else:
                    print(f"\nNo new entry signal generation this loop cycle (not a new M1 candle). Current time: {current_loop_time.strftime('%H:%M:%S')}")
            # --- END: Intelligent Entry Signal Generation ---

            # --- Main Loop Sleep ---
            print(f"\nAnalysis cycle complete. Sleeping for {BOT_LOOP_INTERVAL_SECONDS} seconds...")
            time.sleep(BOT_LOOP_INTERVAL_SECONDS)

    except KeyboardInterrupt:
        print("\nBot interrupted by user (KeyboardInterrupt).")
    except Exception as e:
        print(f"\nAn unexpected error occurred in the main loop: {e}")
    finally:
        print("Shutting down connection to MT5 terminal.")
        if iris_mt5_client:
            iris_mt5_client.shutdown()
            print("Sending shutdown command...")

# --- Run the Bot ---
if __name__ == "__main__":
    run_trading_bot()

In [ ]:
# ===================================================================
# --- Special Debug Cell to Isolate Data Fetching Issues ---
# ===================================================================
print("--- Starting Data Fetch Debug Test ---")

# We use functions from your Cell #2, so make sure it has been run.
if 'connect_and_login' not in globals():
    print("❌ Please run your Cell #1 and Cell #2 first to define the necessary functions.")
else:
    # Establish a temporary, clean connection for this test
    test_client = connect_and_login()

    if test_client:
        try:
            # --- Test Parameters ---
            symbol_to_test = "EURUSD.sml"
            timeframe_to_test = MetaTrader5.TIMEFRAME_H1
            bars_to_request = 10 # Using a small, simple request

            print("\n" + "="*50)
            print(f"--- [METHOD 1] Testing 'copy_rates_from_pos' (The Current Method) ---")
            
            rates_pos = test_client.copy_rates_from_pos(symbol_to_test, timeframe_to_test, 0, bars_to_request)
            
            if rates_pos is not None:
                print(f"✅ Result from copy_rates_from_pos: Got {len(rates_pos)} bars.")
            else:
                print(f"❌ Result from copy_rates_from_pos: Received None.")
            
            error_code, error_message = test_client.last_error()
            print(f"   MT5 Last Error after this call: Code {error_code} - {error_message}")
            print("="*50)


            print("\n" + "="*50)
            print(f"--- [METHOD 2] Testing 'copy_rates_from' (The ALTERNATIVE Method) ---")
            
            # This alternative method uses a start date instead of a position.
            from datetime import datetime, timedelta
            utc_from = datetime.utcnow() - timedelta(days=10)
            
            rates_date = test_client.copy_rates_from(symbol_to_test, timeframe_to_test, utc_from, bars_to_request)
            
            if rates_date is not None:
                print(f"✅ Result from copy_rates_from: Got {len(rates_date)} bars.")
            else:
                print(f"❌ Result from copy_rates_from: Received None.")

            error_code, error_message = test_client.last_error()
            print(f"   MT5 Last Error after this call: Code {error_code} - {error_message}")
            print("="*50)

        except Exception as e:
            print(f"An exception occurred during the test: {e}")
        finally:
            print("\n--- Test complete. Shutting down test connection. ---")
            test_client.shutdown()
    else:
        print("Could not establish a connection for the test.")